In [1]:
import requests
import pandas as pd
import json
import Keys
import time
import pickle
import re
from rauth import OAuth1Service

In [2]:
def get_musix_genres(key):
    url = 'https://api.musixmatch.com/ws/1.1/'
    sub_url = 'music.genres.get'
    params = {
        'apikey': key
    }
    
    musix_genres_json = requests.get(url + sub_url, params = params).json()['message']['body']['music_genre_list']
    
    musix_genres = []
    
    for dictionary in musix_genres_json:
        genre = dictionary['music_genre']
        musix_genres.append({'genre_id': genre['music_genre_id'],
                             'genre_name': genre['music_genre_vanity']})
    
    return musix_genres

In [11]:
def get_musix_track_info_by_genre(genres, key, file, id_limit = 5000):
    url = 'https://api.musixmatch.com/ws/1.1/'
    sub_url = 'track.search'
    
    for genre in genres:
        params = {
            'apikey': key,
            'q_track': '*',
            'f_music_genre_id': genre['genre_id'],
            'f_has_lyrics': 1,
            'f_lyrics_language': 'en',
            'page_size': 100,
            'page': 1
        }
        
        num_tracks = requests.get(url + sub_url, params = params).json()['message']['header']['available']
        page_limit = min(num_tracks, id_limit)
        page_max = (page_limit // 100) + 1
        pages = range(2, page_max)
        
        print('{}: Retrieving {} ids in {} pages'.format(genre['genre_name'], page_limit, page_max - 1))
        print('Retrieving page 1 of {}'.format(page_max - 1))
        
        tracks = requests.get(url + sub_url, params = params).json()['message']['body']['track_list']
        
        with open(file, 'a') as f:
            track_info = ["{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(track['track']['track_id'],
                                                                    track['track']['track_name'],
                                                                    genre['genre_id'],
                                                                    genre['genre_name'],
                                                                    track['track']['album_id'],
                                                                    track['track']['album_name'],
                                                                    track['track']['artist_id'],
                                                                    track['track']['artist_name'])
                          for track in tracks]
                         
            f.writelines(track_info)

        for page in pages:
            time.sleep(1.0)
            params['page'] = page
            print('Retrieving page {} of {}'.format(page, page_max - 1))
            tracks = requests.get(url + sub_url, params = params).json()['message']['body']['track_list']
            
            with open(file, 'a') as f:
                track_info = ["{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(track['track']['track_id'],
                                                                        track['track']['track_name'],
                                                                        genre['genre_id'],
                                                                        genre['genre_name'],
                                                                        track['track']['album_id'],
                                                                        track['track']['album_name'],
                                                                        track['track']['artist_id'],
                                                                        track['track']['artist_name'])
                              for track in tracks]

                f.writelines(track_info)
    
        print('Retrieved {} ids for genre {}'.format(page_limit, genre['genre_name']))
    
    return

In [4]:
musixmatch_url = 'https://api.musixmatch.com/ws/1.1/'
musixmatch_key = Keys.musixmatch_key

In [5]:
with open('../data/raw/genres_dict.pickle', 'rb') as f:
    musix_genres = pickle.load(f)

In [6]:
musix_genres

[{'genre_id': 34, 'genre_name': 'Music'},
 {'genre_id': 1210, 'genre_name': 'Blues-Acoustic-Blues'},
 {'genre_id': 1007, 'genre_name': 'Blues-Chicago-Blues'},
 {'genre_id': 1009, 'genre_name': 'Blues-Classic-Blues'},
 {'genre_id': 1010, 'genre_name': 'Blues-Contemporary-Blues'},
 {'genre_id': 1011, 'genre_name': 'Blues-Country-Blues'},
 {'genre_id': 1012, 'genre_name': 'Blues-Delta-Blues'},
 {'genre_id': 1013, 'genre_name': 'Blues-Electric-Blues'},
 {'genre_id': 1167, 'genre_name': 'Comedy-Novelty'},
 {'genre_id': 1171, 'genre_name': 'Comedy-Standup-Comedy'},
 {'genre_id': 1014, 'genre_name': "Children's-Music-Lullabies"},
 {'genre_id': 1015, 'genre_name': "Children's-Music-Sing-Along"},
 {'genre_id': 1016, 'genre_name': "Children's-Music-Stories"},
 {'genre_id': 1024, 'genre_name': 'Classical-Impressionist'},
 {'genre_id': 1025, 'genre_name': 'Classical-Medieval-Era'},
 {'genre_id': 1026, 'genre_name': 'Classical-Minimalism'},
 {'genre_id': 1027, 'genre_name': 'Classical-Modern-Era'},

In [13]:
genre_ids_to_keep = [2, 6, 11, 14, 15, 17, 18, 20, 22, 24, 1149, 1152]
# top_level_genres = sorted([x for x in musix_genres if x['genre_id'] < 54], key = lambda x: x['genre_id'])
# genre_ids_to_drop = [0, 3, 4, 9, 10, 16, 25, 27, 28, 29, 30, 34, 50, 51, 52, 53]
# genres_to_extract = [x for x in top_level_genres if x['genre_id'] not in genre_ids_to_drop]
genres_to_extract = [x for x in musix_genres if x['genre_id'] in genre_ids_to_keep]

In [15]:
genres_to_extract[5:]

[{'genre_id': 14, 'genre_name': 'Pop'},
 {'genre_id': 15, 'genre_name': 'R-B-Soul'},
 {'genre_id': 17, 'genre_name': 'Dance'},
 {'genre_id': 18, 'genre_name': 'Hip-Hop-Rap'},
 {'genre_id': 20, 'genre_name': 'Alternative'},
 {'genre_id': 22, 'genre_name': 'Christian-Gospel'},
 {'genre_id': 24, 'genre_name': 'Reggae'}]

In [16]:
file = '../data/raw/musix_match_track_data_with_artists_and_albums-02.tsv'
# get_musix_track_info_by_genre([genres_to_extract[0]], musixmatch_key, id_limit = 10_000)
# get_musix_track_info_by_genre(genres_to_extract[1:], musixmatch_key, id_limit = 10_000)
get_musix_track_info_by_genre(genres_to_extract[5:], musixmatch_key, file, id_limit = 10_000)

Pop: Retrieving 10000 ids in 100 pages
Retrieving page 1 of 100
Retrieving page 2 of 100
Retrieving page 3 of 100
Retrieving page 4 of 100
Retrieving page 5 of 100
Retrieving page 6 of 100
Retrieving page 7 of 100
Retrieving page 8 of 100
Retrieving page 9 of 100
Retrieving page 10 of 100
Retrieving page 11 of 100
Retrieving page 12 of 100
Retrieving page 13 of 100
Retrieving page 14 of 100
Retrieving page 15 of 100
Retrieving page 16 of 100
Retrieving page 17 of 100
Retrieving page 18 of 100
Retrieving page 19 of 100
Retrieving page 20 of 100
Retrieving page 21 of 100
Retrieving page 22 of 100
Retrieving page 23 of 100
Retrieving page 24 of 100
Retrieving page 25 of 100
Retrieving page 26 of 100
Retrieving page 27 of 100
Retrieving page 28 of 100
Retrieving page 29 of 100
Retrieving page 30 of 100
Retrieving page 31 of 100
Retrieving page 32 of 100
Retrieving page 33 of 100
Retrieving page 34 of 100
Retrieving page 35 of 100
Retrieving page 36 of 100
Retrieving page 37 of 100
Retrievi

Retrieving page 7 of 100
Retrieving page 8 of 100
Retrieving page 9 of 100
Retrieving page 10 of 100
Retrieving page 11 of 100
Retrieving page 12 of 100
Retrieving page 13 of 100
Retrieving page 14 of 100
Retrieving page 15 of 100
Retrieving page 16 of 100
Retrieving page 17 of 100
Retrieving page 18 of 100
Retrieving page 19 of 100
Retrieving page 20 of 100
Retrieving page 21 of 100
Retrieving page 22 of 100
Retrieving page 23 of 100
Retrieving page 24 of 100
Retrieving page 25 of 100
Retrieving page 26 of 100
Retrieving page 27 of 100
Retrieving page 28 of 100
Retrieving page 29 of 100
Retrieving page 30 of 100
Retrieving page 31 of 100
Retrieving page 32 of 100
Retrieving page 33 of 100
Retrieving page 34 of 100
Retrieving page 35 of 100
Retrieving page 36 of 100
Retrieving page 37 of 100
Retrieving page 38 of 100
Retrieving page 39 of 100
Retrieving page 40 of 100
Retrieving page 41 of 100
Retrieving page 42 of 100
Retrieving page 43 of 100
Retrieving page 44 of 100
Retrieving page

Retrieving page 13 of 100
Retrieving page 14 of 100
Retrieving page 15 of 100
Retrieving page 16 of 100
Retrieving page 17 of 100
Retrieving page 18 of 100
Retrieving page 19 of 100
Retrieving page 20 of 100
Retrieving page 21 of 100
Retrieving page 22 of 100
Retrieving page 23 of 100
Retrieving page 24 of 100
Retrieving page 25 of 100
Retrieving page 26 of 100
Retrieving page 27 of 100
Retrieving page 28 of 100
Retrieving page 29 of 100
Retrieving page 30 of 100
Retrieving page 31 of 100
Retrieving page 32 of 100
Retrieving page 33 of 100
Retrieving page 34 of 100
Retrieving page 35 of 100
Retrieving page 36 of 100
Retrieving page 37 of 100
Retrieving page 38 of 100
Retrieving page 39 of 100
Retrieving page 40 of 100
Retrieving page 41 of 100
Retrieving page 42 of 100
Retrieving page 43 of 100
Retrieving page 44 of 100
Retrieving page 45 of 100
Retrieving page 46 of 100
Retrieving page 47 of 100
Retrieving page 48 of 100
Retrieving page 49 of 100
Retrieving page 50 of 100
Retrieving p

In [139]:
url = 'https://api.musixmatch.com/ws/1.1/'
sub_url = 'track.search'
params = {
    'apikey': musixmatch_key,
    'q_track': '*',
#     'f_music_genre_id': None,
    'f_has_lyrics': 1,
    'f_lyrics_language': 'en',
    'page_size': 100,
    'page': 1
    }

# for genre in genres_to_extract:
#     params['f_music_genre_id'] = genre['genre_id']
#     print('{}: {} available'.format(genre['genre_name'],
#                         requests.get(url + sub_url, params = params).json()['message']['header']['available']))

test = requests.get(url + sub_url, params = params).json()
# test = requests.get(url + sub_url, params = params).json()['message']['body']['track_list']

In [9]:
url = 'https://api.musixmatch.com/ws/1.1/'
sub_url = 'track.search'
params = {
    'apikey': musixmatch_key,
    'q_track': '*',
    'f_music_genre_id': 1095,
    'f_has_lyrics': 1,
    'f_lyrics_language': 'en',
    'page_size': 1,
    'page': 1
}

print('{}: {} available'.format('Christian-Gospel-Metal',
                    requests.get(url + sub_url, params = params).json()['message']['header']['available']))

Christian-Gospel-Metal: 179 available


In [140]:
test

{'message': {'header': {'status_code': 401,
   'execute_time': 0.00035190582275391},
  'body': []}}

In [105]:
track_info = ["{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(track['track']['track_id'],
                                                                    track['track']['track_name'],
                                                                    13,
                                                                    'New-Age',
                                                                    track['track']['album_id'],
                                                                    track['track']['album_name'],
                                                                    track['track']['artist_id'],
                                                                    track['track']['artist_name'])
                          for track in test]

In [106]:
track_info

['52786577\tIsland Rhythms - Relaxing Tropical Shoreline\t13\tNew-Age\t17034085\tWeekend Getaway (Nature Sound With Music)\t26396438\tNaturescape Music\n',
 '69646327\tWhat You Mean to Me\t13\tNew-Age\t19179579\t50 Musak Tracks\t27558642\tBackground Music Soundscape\n',
 '51629081\tBeautiful Night\t13\tNew-Age\t16913776\tVoyage, Vol. 2\t26325478\tBlue Synthax\n',
 '82793288\tMiracle Worker\t13\tNew-Age\t20758691\tAwesome Nature Sounds – Meditation Music for Deep Concentration, Pure Mind, Relaxation Techniques for Anxiety, Motivation, Calming Music for Mental Inspiration\t28663090\tAwesome Nature Sounds Ensemble\n',
 "80167262\tLes six couleurs de l'amour\t13\tNew-Age\t20448298\tLove\t28466590\tLeo Bormans\n",
 '68256576\tKarma\t13\tNew-Age\t18989768\tKarma\t27025600\tClaudio Barria Casanueva\n',
 '98206876\t달님과 별님의 속삭임 The Whisper of Moon And Star\t13\tNew-Age\t21884509\t숲속의 아기 자장가 Lullaby In the Forest\t30701495\tPink Music\n',
 "48101364\tDreamin' In the Water\t13\tNew-Age\t16543677\

In [ ]:
# genre = {'genre_id': 13, 'genre_name': 'New-Age'}

In [ ]:
# with open('../data/raw/musix_match_track_data_with_artists_and_albums.tsv', 'a') as f:
#     track_info = ["{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(track['track']['track_id'],
#                                                                             track['track']['track_name'],
#                                                                             genre['genre_id'],
#                                                                             genre['genre_name'],
#                                                                             track['track']['album_id'],
#                                                                             track['track']['album_name'],
#                                                                             track['track']['artist_id'],
#                                                                             track['track']['artist_name'])
#                   for track in test]
#     f.writelines(track_info)

In [ ]:
# track_info

In [ ]:
# musix_genres = get_musix_genres(musixmatch_key)

# with open('../data/raw/genres_dict.pickle', 'wb') as f:
#     pickle.dump(musix_genres, f)

In [ ]:
with open('../data/raw/musix_match_track_data.csv', 'r') as f:
    csv_data = f.read()

In [ ]:
csv_data_split = csv_data.split('\n')
    

In [ ]:
for index, line in enumerate(csv_data_split):
    line_split = line.split(',')
#     csv_data_split[index] = re.sub(r'([A-Za-z][\w, ]+),', '\"' + "\g<1>" + '\",', line)
    
    if len(line_split) > 4:
        movie_title = "\"{}\"".format(' '.join(line_split[1:len(line_split) - 2]))
        csv_data_split[index] = ','.join([line_split[0], movie_title, line_split[-2], line_split[-1]])

In [ ]:
csv_data_split[24]

In [ ]:
with open('../data/raw/musix_match_track_data_cleaned.csv', 'w') as f:
    for line in csv_data_split:
        f.write(line + '\n')

In [9]:
df = pd.read_csv('../data/raw/musix_match_track_data_cleaned.csv', header = None)
df.columns = ['title_id', 'title_name', 'genre_id', 'genre_name']

In [76]:
df.genre_name.value_counts()

Blues               10000
New-Age             10000
Rock                10000
Reggae              10000
Electronic           9999
Dance                9999
Hip-Hop-Rap          9999
World                9999
Christian-Gospel     9999
R-B-Soul             9999
Holiday              9998
Jazz                 9997
Vocal                9996
Country              9995
Alternative          9993
Pop                  9989
Latin                8000
Classical            6899
Name: genre_name, dtype: int64

In [130]:
df.loc[df.genre_name == 'Pop']

,title_id,title_name,genre_id,genre_name
74888,78395374,Ты не моё,14,Pop
74889,59701326,Back To December (Vocal Version),14,Pop
74890,62708557,Writing On the Wall,14,Pop
74891,79815007,Do You Know the Way to San José,14,Pop
74892,76570084,Pink and Blue,14,Pop
...,...,...,...,...
84872,128775234,Radio (Radio Edit),14,Pop
84873,107413016,Last Kiss 2006,14,Pop
84874,100457346,Paranoia,14,Pop
84875,72910894,Plane Ride,14,Pop


In [4]:
df2 = pd.read_csv('../data/raw/musix_match_track_data_with_artists_and_albums.tsv', sep = '\t', header = None)
df2.columns = ['title_id', 'title_name', 'genre_id', 'genre_name',
               'album_id', 'album_name', 'artist_id', 'artist_name']

In [5]:
df2.genre_name.value_counts()

Rock-Death-Metal-Black-Metal    10000
Rock-Hard-Rock                  10000
Blues                           10000
Jazz                             9999
Country                          9997
Pop                              1400
Name: genre_name, dtype: int64

In [28]:
def scrape_song_lyrics(df, key, file):
    genius = Genius(key)
    
    for row in df.index:
        title = df.iloc[row].title_name
        artist = df.iloc[row].artist_name
        try:
            song = genius.search_song(title, artist)
        except Exception:
            with open('../data/raw/exception-log.txt', 'a') as f:
                f.write('{}, {}, {}'.format(str(type(Exception)), title, artist))
            continue
            
        if song is None:
            continue
        
        song_split = song.lyrics.split('\n')
        song_split_no_empty_headers = [x for x in song_split if '(' not in x and x != '' and '[' not in x]
        song_scraped_artefacts_removed = []

        for line in song_split_no_empty_headers:
            song_scraped_artefacts_removed.append(re.sub(r'([A-Z]?[a-z]+)[A-Z][\w\W]+', '\g<1>', line))

        lyrics = '|'.join(song_scraped_artefacts_removed)

        with open(file, 'a') as f:
            f.write('{}\t{}\t{}\n'.format(lyrics, title, artist))
            
    return

In [172]:
rock_death_black = df2.loc[df2.genre_name == 'Rock-Death-Metal-Black-Metal']

In [170]:
rock_death_black

,title_id,title_name,genre_id,genre_name,album_id,album_name,artist_id,artist_name
0,8401936,Onward to Die,1149,Rock-Death-Metal-Black-Metal,10948290,Scandinavian Warmachine,168822,Paganizer
1,119125944,Illumina,1149,Rock-Death-Metal-Black-Metal,24582909,Humagination,217894,Exlibris
2,3602021,History Bodies,1149,Rock-Death-Metal-Black-Metal,10514844,Abominable,136857,Amagortis
3,6394110,An Eternal Dark Horizon,1149,Rock-Death-Metal-Black-Metal,10761256,An Eternal Dark Horizon,234614,Throne Of Katarsis
4,8401931,An Icon for the Damned,1149,Rock-Death-Metal-Black-Metal,10948290,Scandinavian Warmachine,168822,Paganizer
...,...,...,...,...,...,...,...,...
9995,70232335,Radiation Damage,1149,Rock-Death-Metal-Black-Metal,19270939,Wrought From Anguish,25446897,Human Debris
9996,79417522,Devourer of Worlds,1149,Rock-Death-Metal-Black-Metal,20359725,Rage of the Bloodbeast,89182,Debauchery
9997,163995659,Atheist,1149,Rock-Death-Metal-Black-Metal,31176568,Atheist,28422604,Warshipper
9998,156708372,Immensity,1149,Rock-Death-Metal-Black-Metal,29869359,Silent Witness,36506410,Aeolian


In [198]:
rock_hard_rock = df2.loc[df2.genre_name == 'Rock-Hard-Rock'].reset_index().drop('index', axis = 1)

In [199]:
rock_hard_rock

,title_id,title_name,genre_id,genre_name,album_id,album_name,artist_id,artist_name
0,6594096,Redemptive Soul,1152,Rock-Hard-Rock,10779044,Stranger,326145,Ambitions
1,75833960,Death Rock,1152,Rock-Hard-Rock,19873499,Let's Get Serious,469513,Outloud
2,107807649,Run Baby Run,1152,Rock-Hard-Rock,22998451,Dead Ghost Run,31570758,External Use
3,83212274,Bridge of Regret,1152,Rock-Hard-Rock,20783863,Bridge of Regret,28687393,Protheus
4,70940495,Blacklisted,1152,Rock-Hard-Rock,19365867,Motheredland,27658099,Blakc
...,...,...,...,...,...,...,...,...
9995,138328788,Sororicide,1152,Rock-Hard-Rock,26996595,The Entity,27807316,Sororicide
9996,104958817,Bite the Dust,1152,Rock-Hard-Rock,22603355,The Messenger,68102,Casey Jones
9997,45967539,On Symmetry Pebbles,1152,Rock-Hard-Rock,16322526,"Dance a While, Upset",46412,Settlefish
9998,164242615,Bite the Bullet (United),1152,Rock-Hard-Rock,31228464,The Ultimate Collection Box Set,13293,Harem Scarem


In [7]:
blues = df2.loc[df2.genre_name == 'Blues'].reset_index().drop('index', axis = 1)

In [8]:
blues

,title_id,title_name,genre_id,genre_name,album_id,album_name,artist_id,artist_name
0,47370391,Telephone Wires,2,Blues,16470466,Two To The Chest To Remember One To The Head T...,53617,Feeling Left Out
1,76806842,New York City Dry,2,Blues,20033902,Baby Please Don't Go,6086,Johnny Copeland
2,75175204,Passing Fancy,2,Blues,19808445,Passing Fancy,39802,Vaughn Monroe
3,50184104,I've Got Ringht to Love My Baby,2,Blues,16760431,King of the Blues,8876,B.B. King
4,57191935,Down The Road A Piece,2,Blues,17490237,Playboy Boogie,213471,Big Town Playboys
...,...,...,...,...,...,...,...,...
9995,876058,Up the Line,2,Blues,10292998,Blue Moon,13685,Robben Ford
9996,47590589,Long Time,2,Blues,16491822,Country Blues,1635,Lightnin' Hopkins
9997,70357462,Lonesome (Remastered),2,Blues,19290973,All the Best Songs (Fantastic Masterpieces Rem...,15841,Big Bill Broonzy
9998,114658175,Look At My Baby,2,Blues,23958084,The Chess Box,4292,Bo Diddley


In [12]:
jazz = df2.loc[df2.genre_name == 'Jazz'].reset_index().drop('index', axis = 1)

In [13]:
jazz

,title_id,title_name,genre_id,genre_name,album_id,album_name,artist_id,artist_name
0,71464952,Game 4 the Angels,11,Jazz,19448155,Jazz Na Ulicach,242194,Maria Sadowska
1,46025884,"A Swede,an American, and a Half-breed",11,Jazz,16328231,Vadan,25968035,Vadan
2,8105097,Amen,11,Jazz,10921562,Reference,434682,Piotr Baron
3,47712398,Mauritanie,11,Jazz,16504574,Turbulent silence,198739,Bertrand Renaudin
4,9718036,Brown Sunlight,11,Jazz,11054214,Waiting Room,26873,Addison Groove Project
...,...,...,...,...,...,...,...,...
9994,194018393,Popular Loser,11,Jazz,37160347,Popular Loser: The Jazz Trap L.P.,31426537,G-therz
9995,184007520,Missed Calls from Shanghai,11,Jazz,34880741,Mr. Misdemeanor,33745767,Reece Geller
9996,157640350,Tea N Toast (Interlude),11,Jazz,30048256,"Respiration, Vol. 1",34443499,Fika
9997,205257616,Sin,11,Jazz,40874895,The Abyss,41748822,Velvet Lune


In [20]:
df3 = pd.read_csv('../data/raw/musix_match_track_data_with_artists_and_albums-02.tsv', sep = '\t', header = None)
df3.columns = ['title_id', 'title_name', 'genre_id', 'genre_name',
               'album_id', 'album_name', 'artist_id', 'artist_name']

In [21]:
df3

,title_id,title_name,genre_id,genre_name,album_id,album_name,artist_id,artist_name
0,78395374,Ты не моё,14,Pop,20229239.0,Ты не моё,52534.0,Maria
1,59701326,Back To December (Vocal Version),14,Pop,17782450.0,Karaoke: Taylor Swift (In the Style of Taylor ...,24830063.0,Starlite Karaoke
2,62708557,Writing On the Wall,14,Pop,18175214.0,Cosmology,25960755.0,Robin Stanley
3,79815007,Do You Know the Way to San José,14,Pop,20409871.0,"Stingray Music - Pop Hits of 1968, Vol. 1",24504977.0,Stingray Music
4,76570084,Pink and Blue,14,Pop,20002957.0,Fiebre Tagadá,389211.0,Picnic Kibun
...,...,...,...,...,...,...,...,...
69633,72752997,Money Can't Buy Fren,24,Reggae,19598862.0,Reggae Love Songs & Other Things,27815668.0,Vybz Kartel aka Addi Innocent
69634,237570157,Walls And Bridges,24,Reggae,51849635.0,Walls And Bridges,36712450.0,John Budrevo
69635,59795665,Fly,24,Reggae,17791948.0,Kevin Batchelor's Grand Concourse,24616499.0,Kevin Batchelor
69636,97928715,My Desire,24,Reggae,21860625.0,Memories By The Score Vol. 3,3065.0,John Holt


In [22]:
df3.genre_name.value_counts()

Reggae              10000
Hip-Hop-Rap          9999
Christian-Gospel     9999
Dance                9998
Pop                  9992
R-B-Soul             9834
Alternative          9815
Idiot Pilot             1
Name: genre_name, dtype: int64

In [26]:
from lyricsgenius import Genius

In [27]:
genius_access_token = Keys.genius_access_token

In [23]:
reggae = df3.loc[df3.genre_name == 'Reggae'].reset_index().drop('index', axis = 1)

In [24]:
reggae

,title_id,title_name,genre_id,genre_name,album_id,album_name,artist_id,artist_name
0,6854448,Mama Told Me,24,Reggae,10802894.0,The Boss Harmony Sessions,1457.0,The Slackers
1,19405588,'til the End,24,Reggae,14305585.0,I Revolutionary,14349754.0,RastaMiles
2,19405579,Numbered,24,Reggae,14305585.0,I Revolutionary,14349754.0,RastaMiles
3,78671209,What Is It?,24,Reggae,20261843.0,Part One,137827.0,Indios Bravos
4,61494273,Longtime,24,Reggae,18014604.0,Fearless,105969.0,Lyricson
...,...,...,...,...,...,...,...,...
9995,72752997,Money Can't Buy Fren,24,Reggae,19598862.0,Reggae Love Songs & Other Things,27815668.0,Vybz Kartel aka Addi Innocent
9996,237570157,Walls And Bridges,24,Reggae,51849635.0,Walls And Bridges,36712450.0,John Budrevo
9997,59795665,Fly,24,Reggae,17791948.0,Kevin Batchelor's Grand Concourse,24616499.0,Kevin Batchelor
9998,97928715,My Desire,24,Reggae,21860625.0,Memories By The Score Vol. 3,3065.0,John Holt


In [ ]:
file = '../data/raw/reggae_song_lyrics.tsv'
scrape_song_lyrics(reggae, genius_access_token, file)

Searching for "Mama Told Me" by The Slackers...
Done.
Searching for "'til the End" by RastaMiles...
No results found for: ''til the End RastaMiles'
Searching for "Numbered" by RastaMiles...
Done.
Searching for "What Is It?" by Indios Bravos...
Done.
Searching for "Longtime" by Lyricson...
Done.
Searching for "Dark Forces" by RastaMiles...
Done.
Searching for "Rimshot" by USCB Allstars...
No results found for: 'Rimshot USCB Allstars'
Searching for "January Sky" by Bonsai Nation...
No results found for: 'January Sky Bonsai Nation'
Searching for "Jump Up" by CarlTeh...
No results found for: 'Jump Up CarlTeh'
Searching for "I Don't Know" by Sam and the Stylees...
Done.
Searching for "You and I (Radio Edit)" by Dani Girl & Various Artists...
Done.
Searching for "No Fear" by Mouthwash...
Done.
Searching for "Sinking In the Mist" by The Mighty Three's...
Done.
Searching for "Belly of Lies" by James Shook...
Done.
Searching for "Rockaz Jam" by Father Psalms Studio & Mike Love...
No results fou

No results found for: 'Real Time Lyrics for the Playlist Scratchylus'
Searching for "Truly / Crazy Baldhead" by Geoffrey Chambers & Dread In Brasil...
No results found for: 'Truly / Crazy Baldhead Geoffrey Chambers & Dread In Brasil'
Searching for "Wings of a Dove" by The Hyltonaires...
No results found for: 'Wings of a Dove The Hyltonaires'
Searching for "Come Back" by Askers...
Done.
Searching for "Concrete Jungle (Live)" by Diamusk...
No results found for: 'Concrete Jungle (Live) Diamusk'
Searching for "Freedom (Live)" by Diamusk...
No results found for: 'Freedom (Live) Diamusk'
Searching for "Answering Machine" by The Allstonians...
No results found for: 'Answering Machine The Allstonians'
Searching for "Your Eyes" by I-Octane...
Done.
Searching for "How Does She Feel" by Good Vibe Styla...
Done.
Searching for "T-Bone Song" by Club99...
No results found for: 'T-Bone Song Club99'
Searching for "Badmind Fi the Year" by I-Octane...
No results found for: 'Badmind Fi the Year I-Octane'


Done.
Searching for "We're Dying" by Oz One...
Done.
Searching for "Good Time" by Joe Tolo...
Done.
Searching for "Woman Is Extasy" by Colorblade...
No results found for: 'Woman Is Extasy Colorblade'
Searching for "Dust and Tears" by Romy.K...
Done.
Searching for "Make It" by Anzola...
Done.
Searching for "Don't Drop Out" by Pablo Moses...
Done.
Searching for "Life Anno Game" by Randy Dee...
Done.
Searching for "Blind Guides" by Joshua Alo...
Done.
Searching for "Sort of Invisible (Live)" by The Third and the Mortal...
Done.
Searching for "Mt Slogans" by Maroon Town...
Done.
Searching for "Agatha" by The Busters...
Done.
Searching for "Treading Around The Land" by Leoes de Israel...
Done.
Searching for "Take You Home" by Al Pancho...
Done.
Searching for "Whorshipping Your Love" by Senzo...
Done.
Searching for "Asking" by Winning Jah...
Done.
Searching for "Like a Lioness" by Leah Rosier...
Done.
Searching for "Land Of My Birth" by Fab 5...
Done.
Searching for "Bells of Hate" by Hornsma

Done.
Searching for "Berlin Yard (Reggae Remix)" by Boone Chatta...
No results found for: 'Berlin Yard (Reggae Remix) Boone Chatta'
Searching for "Trippin'" by No Parking Anytime...
Done.
Searching for "Again" by Water...
Done.
Searching for "Welcome to the Trap" by T Que...
Done.
Searching for "Stand By Me" by Black Reggae...
Done.
Searching for "Upside Down (City on a Hilltop)" by Christafari...
Done.
Searching for "Women Rulers" by Lord Laro...
Done.
Searching for "On Another Beat of a Drum (Remix)" by Lucy Monciel...
No results found for: 'On Another Beat of a Drum (Remix) Lucy Monciel'
Searching for "Sip This Life" by Lucy Monciel...
No results found for: 'Sip This Life Lucy Monciel'
Searching for "Misery" by Olivier J Merault...
No results found for: 'Misery Olivier J Merault'
Searching for "Loving Arms" by Yvonne Curtis...
No results found for: 'Loving Arms Yvonne Curtis'
Searching for "The Children Are Crying" by Alton Ellis...
Done.
Searching for "Everybody Gwan Ska" by Mykal 

Done.
Searching for "Exception to the Rule" by Stone Senses...
Done.
Searching for "Copycat (feat. TJ Schaper)" by Metasequoia...
No results found for: 'Copycat (feat. TJ Schaper) Metasequoia'
Searching for "Drip Dry" by Michaelangelo Wolfe...
No results found for: 'Drip Dry Michaelangelo Wolfe'
Searching for "Inna Di Dance" by Stxnga...
No results found for: 'Inna Di Dance Stxnga'
Searching for "Curriculum Vitae" by Harijan...
No results found for: 'Curriculum Vitae Harijan'
Searching for "Bees 'N Ez" by Harijan...
No results found for: 'Bees 'N Ez Harijan'
Searching for "Y3H MI" by Naab...
No results found for: 'Y3H MI Naab'
Searching for "My Generation" by Rude Pride...
Done.
Searching for "Bed of Roses" by Deebzlenuz...
Done.
Searching for "Love Is Life" by Ken Erics...
Done.
Searching for "Moon Cruisin'" by Howard Til Midnight...
Done.
Searching for "My Way" by Olufela...
No results found for: 'My Way Olufela'
Searching for "Fresh Lemonade" by GL Shawn...
No results found for: 'Fr

...

No results found for: 'Now I Know (Live at Sugarshack Sessions) Of Good Nature'
Searching for "What Can I Do" by Of Good Nature...
Done.
Searching for "Trap All Day" by mayalbi feat. Phankystar...
No results found for: 'Trap All Day mayalbi feat. Phankystar'
Searching for "Heaven" by Emmanuel Quiah...
Done.
Searching for "You Save Me" by Terry Linen...
Done.
Searching for "Stand Together" by Gary Guitz...
Done.
Searching for "Dear God" by ConfidenXX feat. Sponge Music...
Done.
Searching for "Respect, Show I Some Love (feat. Haile Maskel)" by Skyhigh the Mau Mau...
No results found for: 'Respect, Show I Some Love (feat. Haile Maskel) Skyhigh the Mau Mau'
Searching for "All I Want (Major Riddim)" by Rockcity...
No results found for: 'All I Want (Major Riddim) Rockcity'
Searching for "Mother NaJah" by Rina Butterfly...
Done.
Searching for "Bring Me Home" by The B-Side Band...
Done.
Searching for "KONA" by Sugar Lee...
Done.
Searching for "I'll Be Right Here" by Cassie Deborah...
No result

Done.
Searching for "Holy Mount Zion (Remix)" by Alborosie...
No results found for: 'Holy Mount Zion (Remix) Alborosie'
Searching for "When I'm Gone" by Miniirosee...
No results found for: 'When I'm Gone Miniirosee'
Searching for "On the Road" by Protoje...
Done.
Searching for "Good Day" by Hoaikane...
Done.
Searching for "Most High" by Jah Mason...
Done.
Searching for "Good Things (Remastered)" by Lucky Dube...
Done.
Searching for "Strip Tease" by Power feat. Chezy...
Done.
Searching for "The Garden" by The Green...
Done.
Searching for "One More Night" by One Drop...
Done.
Searching for "Music Is the Key" by The Congos...
Done.
Searching for "Sharing The Night Together" by Delroy Wilson...
No results found for: 'Sharing The Night Together Delroy Wilson'
Searching for "Things & Time" by Wailing Souls...
Done.
Searching for "Liberation Call" by Groundation...
Done.
Searching for "New Dawn" by Alpha Blondy...
Done.
Searching for "Love Dem / Gallis Anthem" by Vybz Kartel...
Done.
Searchin

Done.
Searching for "Liquid Sunshine" by Biga Ranx...
Done.
Searching for "(I Can’t Help) Falling In Love With You" by The Countdown Singers...
Done.
Searching for "Dancing Mood" by Dennis Brown...
Done.
Searching for "Brand New Second Hand" by Peter Tosh...
Done.
Searching for "Good Times" by Buju Banton Feat. Fat Joe & Beres Hammond...
Done.
Searching for "Fire" by The Black Seeds...
Done.
Searching for "The Die Is Cast" by Nguru...
Done.
Searching for "Tek off Something" by Stiffy...
Done.
Searching for "Drive Me Crazy" by Mr. Easy...
Done.
Searching for "Have Some Class" by Imar Shephard...
Done.
Searching for "Vibes" by Tommy Lee Sparta...
Done.
Searching for "Jump" by Rupee...
Done.
Searching for "HOLD ME" by Rik Rok...
Done.
Searching for "Should I Tell Her" by Rik Rok...
Done.
Searching for "Seriously" by Katchafire...
Done.
Searching for "Little Green Apples" by Everton Blender...
Done.
Searching for "Love Me Easy" by ANORA feat. Stick Figure & Walshy Fire...
Done.
Searching f

No results found for: 'Natural Mystic - Live At Rainbow Theatre, London / 1977 Bob Marley & The Wailers'
Searching for "Cry To Me" by Bob Marley & The Wailers...
Done.
Searching for "Before You Leave" by Dexta Daps...
Done.
Searching for "What Will It Take" by Jah Cure...
Done.
Searching for "Hold You" by Gyptian...
Done.
Searching for "Berlin Everything" by Biga Ranx...
Done.
Searching for "Bun Nu Nu" by Tommy Lee Sparta...
Done.
Searching for "So Real 2.0" by LMK...
Done.
Searching for "Bus Dem Shut (JAD)" by Bob Marley & The Wailers...
No results found for: 'Bus Dem Shut (JAD) Bob Marley & The Wailers'
Searching for "Addicted" by Sammielz...
Done.
Searching for "All I Want" by Sizzla feat. Morgan Heritage...
No results found for: 'All I Want Sizzla feat. Morgan Heritage'
Searching for "All About You" by Konshens...
Done.
Searching for "Africa" by Trilogy Reggae...
No results found for: 'Africa Trilogy Reggae'
Searching for "Rest Your Love on Me" by Perfect...
Done.
Searching for "Ai

Done.
Searching for "Tougher Than Tough" by Derrick Morgan...
Done.
Searching for "Warning" by Jada Kingdom...
Done.
Searching for "Sweet Love" by House of Shem...
Done.
Searching for "All the Small Things" by Skatune Network...
No results found for: 'All the Small Things Skatune Network'
Searching for "I'm Coming Home" by Morgan Heritage...
Done.
Searching for "Never To Late" by Siddy Ranks...
Done.
Searching for "Wake Me Up - Reggae Mix" by Corcovado Frequency...
No results found for: 'Wake Me Up - Reggae Mix Corcovado Frequency'
Searching for "Jah Love Mi" by Konshens...
Done.
Searching for "Never Sober" by Popcaan...
Done.
Searching for "Write Off The Debt" by UB40...
Done.
Searching for "You Baby" by Dancing Mood...
Done.
Searching for "Movin' On" by Glen Washington...
Done.
Searching for "Lively Up Yourself - Live At The Lyceum, London/1975" by Bob Marley & The Wailers...
No results found for: 'Lively Up Yourself - Live At The Lyceum, London/1975 Bob Marley & The Wailers'
Searchi

In [181]:
scrape_song_lyrics(rock_death_black, genius_access_token)

Searching for "Onward to Die" by Paganizer...
Done.
Searching for "Illumina" by Exlibris...
Done.
Searching for "History Bodies" by Amagortis...
Searching for "An Eternal Dark Horizon" by Throne Of Katarsis...
Done.
Searching for "An Icon for the Damned" by Paganizer...
Done.
Searching for "Slaughtered Corpse" by Amagortis...
Done.
Searching for "Lost Children in the World" by Opus Doria...
Done.
Searching for "Gorroneality" by Avulsed...
No results found for: 'Gorroneality Avulsed'
Searching for "Heraldic" by Autumnblaze...
Done.
Searching for "Iced" by Bodyfarm...
Done.
Searching for "Remembrances" by Blessed in Sin...
Done.
Searching for "Soultheft" by Centurian...
Done.
Searching for "Bonesaw Bonanza" by Paganizer...
Done.
Searching for "Departure" by Eye of Solitude...
Done.
Searching for "Departure of the Stellar Fleet Marks the Year One" by Deconstructing Sequence...
No results found for: 'Departure of the Stellar Fleet Marks the Year One Deconstructing Sequence'
Searching for "

Done.
Searching for "Phantoms" by Nominon...
Done.
Searching for "Below All That Is Mortal" by Fragments of Unbecoming...
Done.
Searching for "Fatum (III Fate - Noodlot)" by Ancient Rites...
No results found for: 'Fatum (III Fate - Noodlot) Ancient Rites'
Searching for "He Sells Agony" by Abaddon Incarnate...
Done.
Searching for "Finger on the Trigger" by Voices of Decay...
Done.
Searching for "When the Solstice Reaches the Apogee" by Mythological Cold Towers...
No results found for: 'When the Solstice Reaches the Apogee Mythological Cold Towers'
Searching for "The Angels" by Dying Suffocation...
Done.
Searching for "Requiem on Their Graveth" by Arkha Sva...
Done.
Searching for "Beyond Their Lies" by Wedingoth...
No results found for: 'Beyond Their Lies Wedingoth'
Searching for "Into Red" by For Ruin...
Done.
Searching for "Ethnic Cleansing" by Inner Thought...
Done.
Searching for "Morbid Panzer Batallion" by Paganizer...
No results found for: 'Morbid Panzer Batallion Paganizer'
Search

Done.
Searching for "Death Comes in March" by ToDieFor...
Done.
Searching for "End of Tears" by ToDieFor...
Done.
Searching for "Framed Living Bust" by Castrensis...
Done.
Searching for "Unholy Soul" by Stefano Zani's Project...
No results found for: 'Unholy Soul Stefano Zani's Project'
Searching for "Interlude : Become of Me" by Enchantya...
Done.
Searching for "Disciples of the Hex" by Damnations Hammer...
Done.
Searching for "Hibernation Sun" by Fromhell...
No results found for: 'Hibernation Sun Fromhell'
Searching for "Shedding Liquid Flesh" by Fetocide...
No results found for: 'Shedding Liquid Flesh Fetocide'
Searching for "Cry for Love" by ToDieFor...
Done.
Searching for "Unknown III" by ToDieFor...
No results found for: 'Unknown III ToDieFor'
Searching for "Oblivion: Vision" by ToDieFor...
Done.
Searching for "Letters from Death Row, Pt. 1" by Stefano Zani's Project...
No results found for: 'Letters from Death Row, Pt. 1 Stefano Zani's Project'
Searching for "Insanity's Edge" by

Done.
Searching for "Yet I Breathe" by Eye of Solitude...
Done.
Searching for "Gathering the Stones" by Fen...
Done.
Searching for "March of Demise" by Kormak...
No results found for: 'March of Demise Kormak'
Searching for "Epitaph" by Messiah...
Done.
Searching for "Not A Nightmare" by Artillery...
Done.
Searching for "Lurid Assimilation" by Defeated Sanity...
Done.
Searching for "Divide et Impera" by Lahmia...
Done.
Searching for "Unplug My Brain" by Asylum Pyre...
Done.
Searching for "My Last Call" by Slechtvalk...
Done.
Searching for "It Comes in Whispers" by Leviathan...
Done.
Searching for "Caixa Pensatória" by Dallian...
No results found for: 'Caixa Pensatória Dallian'
Searching for "Broken Mirror" by Ashes of Moon...
Done.
Searching for "Under a Moonlit Sky" by Slechtvalk...
Done.
Searching for "Zealous Procreation" by Hetroertzen...
Done.
Searching for "Wounds of Christ" by Liturgy...
Done.
Searching for "Violence Reigns Supreme" by Winter Of Sin...
Done.
Searching for "Mephis

Done.
Searching for "Horrified by Repulsion" by Avulsed...
Done.
Searching for "Perfidia" by The Kennedy Veil...
Done.
Searching for "Towards The Atmospheric" by Crows As Divine...
Done.
Searching for "Rite of Passage" by Ancestors Blood...
Done.
Searching for "Born but Buried, I Can See the Light" by Intestine Baalism...
Done.
Searching for "Brethren of the Cross" by The Infernal Sea...
Done.
Searching for "Fetal Scabs" by Lividity...
Done.
Searching for "Jackpot" by Pokerface...
Done.
Searching for "The Wolves Have Fed Tonight" by Claret Ash...
No results found for: 'The Wolves Have Fed Tonight Claret Ash'
Searching for "Unchained" by Burden of Grief...
Done.
Searching for "Within the Realm of a Dying Sun (Live)" by Love Like Blood...
Done.
Searching for "Black Swan Events" by I Shalt Become...
Done.
Searching for "Underwatermoon" by Winterhorde...
Done.
Searching for "Take My Hand" by Heavenwood...
Done.
Searching for "Thy Baptism" by Arkha Sva...
Done.
Searching for "Monolith" by E

...

Done.
Searching for "The Key is Inside the Wall" by Darkthrone...
Done.
Searching for "Death Whispered a Lullaby" by Opeth...
Done.
Searching for "War Born (Tri-adverserenade)" by Daath...
Done.
Searching for "The Demons in Me" by Savn...
Done.
Searching for "Revolt" by Gorefest...
Done.
Searching for "Ravishing Grimness" by Darkthrone...
Done.
Searching for "Save Our Souls" by Clawfinger...
Done.
Searching for "Too Late to Call an Ambulance" by Psychonaut 4...
Done.
Searching for "Useless Sacrifice" by Death Decline...
Done.
Searching for "Shit Comes Brown" by Krabathor...
Done.
Searching for "Spawn of Flesh" by God Macabre...
Done.
Searching for "Souls of the Evil Departed" by At the Gates...
Done.
Searching for "Satan Redeemer" by Eosphorus...
No results found for: 'Satan Redeemer Eosphorus'
Searching for "The Link" by Gojira...
Done.
Searching for "No Amen" by Iskald...
Done.
Searching for "Nightmares of an Ether Drinker" by Cradle of Filth...
Done.
Searching for "Moment of Fright"

No results found for: 'Life on Your Knees (2018 Remaster) Neurosis'
Searching for "Plague Bearer" by Claret Ash...
Done.
Searching for "Temptation ov Lost" by Christ Agony...
Done.
Searching for "Essence of Fire" by Claret Ash...
Done.
Searching for "Origin" by Todtgelichter...
No results found for: 'Origin Todtgelichter'
Searching for "Catarctic Cartoons" by Arctic Plateau...
Specified song does not contain lyrics. Rejecting.
Searching for "We Love Because He First Loved Us" by GaidjinN...
Done.
Searching for "Fuigheall" by Cnoc An Tursa...
No results found for: 'Fuigheall Cnoc An Tursa'
Searching for "N.A.F.H." by Selvans...
Done.
Searching for "Summon the Horde" by Asmodeus...
Done.
Searching for "A Handful of Nothing" by Ebony Tears...
Done.
Searching for "The Sixth Bloodletting" by In Reverence...
Done.
Searching for "My Empire" by Deserted Fear...
Done.
Searching for "The Fall of Irminsul" by Rebellion...
Done.
Searching for "My Ulysses" by I Spit Ashes...
Done.
Searching for "Cr

Done.
Searching for "Carthago Delenda Est" by Ancient Rites...
Done.
Searching for "For the Dying Hope" by Vision Eyes...
Done.
Searching for "The Spectre" by Messenger...
Done.
Searching for "Mouth of Madness (Live)" by Tsjuder...
No results found for: 'Mouth of Madness (Live) Tsjuder'
Searching for "Planet Zombie" by Zombie Inc....
Done.
Searching for "Shahanshah" by Viper Solfa...
Done.
Searching for "Bionic Abomination" by Deadborn...
Done.
Searching for "The Newborn" by Furtherial...
Done.
Searching for "Scandinavian Warmachine" by Paganizer...
Done.
Searching for "Speed of God" by Kekal...
Done.
Searching for "Exalting the Shadow Proprietor" by Cara Neir...
Done.
Searching for "Precept of Delator" by Lectern...
No results found for: 'Precept of Delator Lectern'
Searching for "Ruthless Cleansing" by Brutally Deceased...
Done.
Searching for "The Fallen Hero" by Heaven's Scum...
Done.
Searching for "27 Days of Darkness" by Phobocosm...
Done.
Searching for "(We Are The) Doomed" by Ci

Done.
Searching for "Black Widow" by Perséphone...
Done.
Searching for "Ovulation Bitch" by Vendetta...
Done.
Searching for "Last of a Fading Kind" by Deserted Fear...
Done.
Searching for "Still Standing Up" by Bloodhunter...
No results found for: 'Still Standing Up Bloodhunter'
Searching for "Cry. Pray. Die." by Pokerface...
Done.
Searching for "Elisabeth Dane" by The Vision Bleak...
Done.
Searching for "Ashes Remain" by Blaze of Perdition...
Done.
Searching for "Avydia" by Otus...
No results found for: 'Avydia Otus'
Searching for "Mushroom Song" by WAIT...
Done.
Searching for "Nutrient Painting" by Yellow Eyes...
Done.
Searching for "The Awakening Of The Beast" by Death SS...
Done.
Searching for "Driven by Nescience" by Nervecell...
Done.
Searching for "Ageless Existence" by The Ravenous...
Done.
Searching for "Feasting from the Womb" by The Ravenous...
Done.
Searching for "Orgy in Dog's Blood" by The Ravenous...
Done.
Searching for "Perverted Before God" by The Ravenous...
Done.
Sea

Done.
Searching for "Funereal Might" by Arsis...
Done.
Searching for "Clockwise Metamorphosis" by Tomb Mold...
Done.
Searching for "Wicked Fields of Calm" by Leviathan...
Done.
Searching for "Deviant" by Despite Exile...
Done.
Searching for "Sorgen (Sunwheels)" by 1476...
Done.
Searching for "The Valkyries Call" by Stormwarrior...
Done.
Searching for "At the Sound of the Millenium Black Bells - Live" by Enthroned...
Done.
Searching for "Silence" by Catamenia...
Done.
Searching for "Ave Master Lucifer" by Besatt...
Done.
Searching for "Blinding Sun" by Sad Theory...
Done.
Searching for "The Endless Regression of Mind" by Odious Mortem...
Done.
Searching for "Heroes in Godly Blaze" by Adramelech...
Done.
Searching for "Within the Realm of a Dying Sun" by Love Like Blood...
Done.
Searching for "Solace" by Ashbringer...
Done.
Searching for "Golem" by Darkwell...
Done.
Searching for "Night Vision" by Medium...
Done.
Searching for "Flames Of An Endless Disease" by Soulburner...
No results fo

In [189]:
file = '../data/raw/rock_hard_rock_song_lyrics.tsv'
scrape_song_lyrics(rock_hard_rock, genius_access_token, file)

IndexError: single positional indexer is out-of-bounds

In [11]:
file = '../data/raw/blues_song_lyrics.tsv'
scrape_song_lyrics(blues, genius_access_token, file)

Searching for "Telephone Wires" by Feeling Left Out...
Done.
Searching for "New York City Dry" by Johnny Copeland...
Done.
Searching for "Passing Fancy" by Vaughn Monroe...
No results found for: 'Passing Fancy Vaughn Monroe'
Searching for "I've Got Ringht to Love My Baby" by B.B. King...
Done.
Searching for "Down The Road A Piece" by Big Town Playboys...
Done.
Searching for "White Jacket" by Rex Luciferius...
No results found for: 'White Jacket Rex Luciferius'
Searching for "The Box" by Rex Luciferius...
Done.
Searching for "Drivin' Home to You" by Guy Verlinde...
Done.
Searching for "If You Hear Me Cryin'" by Izzy & Chris...
Done.
Searching for "Death Kneel (The New Death)" by Space Bong...
Done.
Searching for "The Rain Don' Fall On Me" by Geoff Muldaur...
No results found for: 'The Rain Don' Fall On Me Geoff Muldaur'
Searching for "Heavy Soul" by The City Hollers...
Done.
Searching for "Home Run" by Celeste...
Done.
Searching for "Star-Shaped Hole" by Hanging Tree...
No results found

Done.
Searching for "I Be's Troubled (Digitally Remastered)" by Muddy Waters...
No results found for: 'I Be's Troubled (Digitally Remastered) Muddy Waters'
Searching for "That Same Thing (Live)" by Muddy Waters...
Done.
Searching for "That's All (Live at AFRS)" by Sister Rosetta Tharpe...
Done.
Searching for "Fine Looking Woman (Digitally Remastered)" by B.B. King...
No results found for: 'Fine Looking Woman (Digitally Remastered) B.B. King'
Searching for "I'm a Black Woman's Man (Remastered)" by Brownie McGhee...
No results found for: 'I'm a Black Woman's Man (Remastered) Brownie McGhee'
Searching for "Crawdad (Remastered)" by Big Bill Broonzy...
No results found for: 'Crawdad (Remastered) Big Bill Broonzy'
Searching for "Down the Road a Piece (Live)" by Big Joe Duskin...
Done.
Searching for "Don't Start Me Talking (Live)" by Peter Green Splinter Group...
Done.
Searching for "My Idea (Remastered)" by Dinah Washington...
Done.
Searching for "Shukin' and Jivin' (Remastered)" by Dinah Wa

Done.
Searching for "John Hardy - Remastered" by Lead Belly...
Done.
Searching for "Alberta (Remastered)" by Lead Belly...
Done.
Searching for "Love Divine" by Crooked Eye Tommy...
Done.
Searching for "Ramblin'" by Dan Stevens...
Done.
Searching for "Don't Ever Let Nobody Drag Your Spirit Down" by Big Daddy Wilson & Doc Fozz...
No results found for: 'Don't Ever Let Nobody Drag Your Spirit Down Big Daddy Wilson & Doc Fozz'
Searching for "Mountain Boogie" by Bjørn Berge...
Done.
Searching for "Stories of a Hero" by Frank Marino & Mahogany Rush...
Done.
Searching for "Belle of the West" by Samantha Fish...
Done.
Searching for "My Baby's an Angel" by B.B. King...
Done.
Searching for "All I Wanna Do Is Love You" by Reno Trihardityo...
No results found for: 'All I Wanna Do Is Love You Reno Trihardityo'
Searching for "Hallelujah I Love You So" by Hugh Laurie...
Done.
Searching for "Walk On the Wild Side / Lou Reed" by Gee Gee & Soluna...
No results found for: 'Walk On the Wild Side / Lou Reed

Done.
Searching for "King For a Day" by Eddie "Cleanhead" Vinson...
No results found for: 'King For a Day Eddie "Cleanhead" Vinson'
Searching for "Killin' Time Blues" by Eric Sardinas...
Done.
Searching for "He Doesn't Live Like the Others" by The Lee Sankey Group...
No results found for: 'He Doesn't Live Like the Others The Lee Sankey Group'
Searching for "Southern Jumbo" by Colin Linden...
Done.
Searching for "Mr. Highvibe" by Brendon Thomas and the Vibes...
No results found for: 'Mr. Highvibe Brendon Thomas and the Vibes'
Searching for "Single Tree Blues" by Roosevelt Sykes...
No results found for: 'Single Tree Blues Roosevelt Sykes'
Searching for "You Gave Nothing" by Mississippi Heat...
Done.
Searching for "Fire Detective Blues" by Roosevelt Sykes...
No results found for: 'Fire Detective Blues Roosevelt Sykes'
Searching for "Devil's Outta Get Me" by Bourbon Blues Ensemble...
No results found for: 'Devil's Outta Get Me Bourbon Blues Ensemble'
Searching for "Possum Up a 'Simmon Tree

Done.
Searching for "Find Balance." by Donte Millz...
Done.
Searching for "Evil Ways" by Psilojam...
No results found for: 'Evil Ways Psilojam'
Searching for "On the Wall / Louise Johnson" by Charley Patton...
Done.
Searching for "Eyes on the Prize (Building the Panama Canal)" by Hendrith Vanlon Smith Jr...
No results found for: 'Eyes on the Prize (Building the Panama Canal) Hendrith Vanlon Smith Jr'
Searching for "John Law Burned Down The Liquor Sto'" by Chris Thomas King...
Done.
Searching for "Mama's Advice" by Peetie Wheatstraw...
No results found for: 'Mama's Advice Peetie Wheatstraw'
Searching for "The Promise True And Grand" by Bukka White...
Done.
Searching for "Heart Shaped Lips" by Triple S...
Done.
Searching for "Play It Right" by Union Grail...
Done.
Searching for "Autumn Blues & Shiddy Weddah" by Marco Akamawa...
No results found for: 'Autumn Blues & Shiddy Weddah Marco Akamawa'
Searching for "So Attracted to You" by Gina Sicilia...
Done.
Searching for "My Grey Pony" by Bi

...

No results found for: 'The Bourgeois Blues (Remastered Version) Leadbelly'
Searching for "Long Lonesome Blues (2472-1)" by Blind Lemon Jefferson...
Done.
Searching for "Long Lonesome Blues #1" by Blind Lemon Jefferson...
Done.
Searching for "A Million Miles Away (Live at Belfast)" by Rory Gallagher...
No results found for: 'A Million Miles Away (Live at Belfast) Rory Gallagher'
Searching for "Little Bittie Gal's Blues" by Big Joe Turner...
Done.
Searching for "Lose Your Money (But Don't Lose Your Mind)" by Delbert McClinton...
No results found for: 'Lose Your Money (But Don't Lose Your Mind) Delbert McClinton'
Searching for "Love March" by The Paul Butterfield Blues Band...
Done.
Searching for "For So Long" by Harrison Flint...
Done.
Searching for "Fine Yesterday" by Robert Cray...
Done.
Searching for "Funny" by Jeroen Claase...
No results found for: 'Funny Jeroen Claase'
Searching for "Big Money Grip" by Jason Elmore & Hoodoo Witch...
No results found for: 'Big Money Grip Jason Elmore

Done.
Searching for "My Baby Been Gone" by Snooky Pryor...
Done.
Searching for "Just Like Me" by Tommy Castro...
Done.
Searching for "It's Too Late Brother" by Little Walter...
Done.
Searching for "John Henry" by Furry Lewis...
Done.
Searching for "Juggernaut" by Frank Marino & Mahogany Rush...
Done.
Searching for "Living a Lie" by Landon Tewers...
Done.
Searching for "Mannish Boy - Live" by Muddy Waters...
Done.
Searching for "A Lover's Question" by Lou Rawls...
Done.
Searching for "Money Blues" by Jeffrey Foucault...
Done.
Searching for "Love Song" by Shemekia Copeland...
No results found for: 'Love Song Shemekia Copeland'
Searching for "Keeps Me Bleeding" by Black River Delta...
Done.
Searching for "Jubilee" by Charlie Parr...
Done.
Searching for "Kingdom Come" by Iron Mike Norton...
Done.
Searching for "Keep Walkin'" by Iron Mike Norton...
Done.
Searching for "Keep Me On The Road" by Karl S. Williams...
Done.
Searching for "So Many Things" by Gabe Bailey...
Done.
Searching for "So 

No results found for: 'Let the Good Times Roll (Live at Coconut Grove/Single Version) B.B. King feat. Bobby "Blue" Bland'
Searching for "Let the Good Times Roll (Remastered)" by Koko Taylor...
Done.
Searching for "Little Wheel" by The Smokin' Joe Kubek Band...
No results found for: 'Little Wheel The Smokin' Joe Kubek Band'
Searching for "Lonely Avenue" by Taj Mahal...
Done.
Searching for "Lonely Avenue" by Colin James...
Done.
Searching for "Lonely Avenue - Live" by The Blues Band...
Done.
Searching for "Let's Have a Ball (Here Am I)" by Ray Charles...
Done.
Searching for "Old Dusty Road" by 20 Year Short Break...
Done.
Searching for "Lifeline" by Hans Theessink...
No results found for: 'Lifeline Hans Theessink'
Searching for "Dust My Broom" by John Hammond...
Done.
Searching for "Postcards From Hell" by The Wood Brothers...
Done.
Searching for "My Road Lies In Darkness" by Charlie Musselwhite...
Done.
Searching for "Man On The Run" by Lowell Fulson...
Done.
Searching for "Manic Depres

Done.
Searching for "Doctor Told Me" by J.J. Cale...
Done.
Searching for "Daughters" by Samantha Fish...
Done.
Searching for "Blues Attack" by Sonny Landreth...
Done.
Searching for "Frisco Town - Remastered" by Memphis Minnie...
No results found for: 'Frisco Town - Remastered Memphis Minnie'
Searching for "'Frisco Town" by Memphis Minnie...
Done.
Searching for "Frigidaire Woman" by Son Seals...
Done.
Searching for "The Fresh Prince of Bel-Air Theme" by samuraiguitarist...
No results found for: 'The Fresh Prince of Bel-Air Theme samuraiguitarist'
Searching for "Ain't No Grave Gonna Hold My Body Down" by Charlie Parr...
Done.
Searching for "A Taste Of The Blues" by Pete Alderton...
Done.
Searching for "Asking Around for You" by Joe Bonamassa...
Done.
Searching for "Brown Eyed Handsome Man (Live)" by Johnny Winter...
Done.
Searching for "He Was There" by James Cotton...
Done.
Searching for "(Don't) Let Me Drown [Acoustic]" by Paul Cargnello...
No results found for: '(Don't) Let Me Drown [

Done.
Searching for "Where Can You Be" by The Smokin' Joe Kubek Band...
No results found for: 'Where Can You Be The Smokin' Joe Kubek Band'
Searching for "Up the Line" by Robben Ford...
Done.
Searching for "Long Time" by Lightnin' Hopkins...
Done.
Searching for "Lonesome (Remastered)" by Big Bill Broonzy...
No results found for: 'Lonesome (Remastered) Big Bill Broonzy'
Searching for "Look At My Baby" by Bo Diddley...
Done.
Searching for "Last Night" by Little Walter...
Done.


In [14]:
file = '../data/raw/jazz_song_lyrics.tsv'
scrape_song_lyrics(jazz, genius_access_token, file)

Searching for "Game 4 the Angels" by Maria Sadowska...
No results found for: 'Game 4 the Angels Maria Sadowska'
Searching for "A Swede,an American, and a Half-breed" by Vadan...
No results found for: 'A Swede,an American, and a Half-breed Vadan'
Searching for "Amen" by Piotr Baron...
Done.
Searching for "Mauritanie" by Bertrand Renaudin...
No results found for: 'Mauritanie Bertrand Renaudin'
Searching for "Brown Sunlight" by Addison Groove Project...
Done.
Searching for "Yüksek Yüksek Tepelere " On the High High Hills"" by Alaturka...
No results found for: 'Yüksek Yüksek Tepelere " On the High High Hills" Alaturka'
Searching for "Hey Rube!" by David Carroll & His Orchestra...
Done.
Searching for "Spangled Banner" by Best Saxophone Tribute Orchestra...
No results found for: 'Spangled Banner Best Saxophone Tribute Orchestra'
Searching for "Arrolo de Toques" by Nani García...
No results found for: 'Arrolo de Toques Nani García'
Searching for "Raising the Bar" by John Niems...
Done.
Search

Done.
Searching for "Blue's Suede Shoes" by Bobby Blue...
Done.
Searching for "The Air That I Breath" by Della Reese...
Done.
Searching for "I Didn't Know What This Time It Was" by Anita O'Day...
Done.
Searching for "Medley: Jealous / Swinging Down the Lane / Breezing Along With the Breeze" by Claude Thornhill and His Orchestra...
No results found for: 'Medley: Jealous / Swinging Down the Lane / Breezing Along With the Breeze Claude Thornhill and His Orchestra'
Searching for "As Long As I Love You (Jeannette)" by Bennie Moten's Kansas City Orchestra...
No results found for: 'As Long As I Love You (Jeannette) Bennie Moten's Kansas City Orchestra'
Searching for "The Main Event" by Daryl Lowery Quartet...
Done.
Searching for "Consider" by The Nylons...
Done.
Searching for "If You Asked Me To" by Studio Sunset...
Done.
Searching for "Jealous / Swinging Down the Lane / Breezing Along With the Breeze" by Gil Evans...
No results found for: 'Jealous / Swinging Down the Lane / Breezing Along Wi

Done.
Searching for "Jilted With Frankie Vaughan" by Alma Cogan...
No results found for: 'Jilted With Frankie Vaughan Alma Cogan'
Searching for "Farrago Piano Fantasy: I. Adagio" by Gary Farr...
No results found for: 'Farrago Piano Fantasy: I. Adagio Gary Farr'
Searching for "Walking High" by Alma Negra...
Done.
Searching for "The Way We Celebrate New Year's" by Gap Mangione...
Done.
Searching for "Lady Be Good" by Phil Wilson...
Done.
Searching for "How Can I Face This Wearied World Alone ? (McRae-Mallory)" by Ethel Waters...
No results found for: 'How Can I Face This Wearied World Alone ? (McRae-Mallory) Ethel Waters'
Searching for "Sugar" by Jamey Aebersold Play-A-Long...
No results found for: 'Sugar Jamey Aebersold Play-A-Long'
Searching for "The Song Is You" by Angela DeNiro...
Done.
Searching for "Earlysville" by King Teddy...
No results found for: 'Earlysville King Teddy'
Searching for "That's All I Need" by Studio Sunset...
Done.
Searching for "You Are the One" by Vaughn Monroe

Done.
Searching for "Trouble All the Way" by Mia Silvas...
Done.
Searching for "My Little Girl" by Spike Jones & His City Slickers...
Done.
Searching for "Fire Flames" by Norine Braun...
Done.
Searching for "Where Or When" by Al Hibbler...
Done.
Searching for "遙かなる影" by acoustic air...
No results found for: '遙かなる影 acoustic air'
Searching for "Swinging Down the Lane" by Richard Maltby...
No results found for: 'Swinging Down the Lane Richard Maltby'
Searching for "Both Sides Now" by Howard Keel...
Done.
Searching for "Sing It Children, Sing It" by Dorothy Collins...
Done.
Searching for "Wedding of Mr and Mrs Swing" by Cab Calloway...
Done.
Searching for "Don't Blame Me" by Martial Solal...
Done.
Searching for "More Fool Me" by Mia Silvas...
Done.
Searching for "That Old Black Magic" by Harry Arnold...
Done.
Searching for "I Want My Lou" by Cowboy Copas...
Done.
Searching for "This Night" by Steve Lawrence...
Done.
Searching for "You're Sensational" by APM Karaoke...
No results found for:

No results found for: 'You're a Sweetheart (Remastered) (From "I Love the Life I Live") Mose Allison'
Searching for "No Smokes Blues (Remastered)" by Sarah Vaughan...
Done.
Searching for "Same As Before (Live)" by Esbjörn Svensson Trio...
No results found for: 'Same As Before (Live) Esbjörn Svensson Trio'
Searching for "Too Funky in Here (Live Version)" by James Brown...
Done.
Searching for "It's Only a Paper Moon (Live) [Remastered]" by Art Tatum...
Done.
Searching for "Zing! Went the Strings of My Heart (Remastered)" by Jimmy Smith...
No results found for: 'Zing! Went the Strings of My Heart (Remastered) Jimmy Smith'
Searching for "Embraceable You (Karaoke In the Style of Frank Sinatra)" by MIDIFine Systems...
No results found for: 'Embraceable You (Karaoke In the Style of Frank Sinatra) MIDIFine Systems'
Searching for "Lamplight (Remastered)" by Della Reese...
No results found for: 'Lamplight (Remastered) Della Reese'
Searching for "Medley of Armstrong Hits - Part 2 (Remastered) [Al

...

Done.
Searching for "Love Like Jazz (feat. J.White & Ashton Vaughn Charles)" by Caira Marie...
No results found for: 'Love Like Jazz (feat. J.White & Ashton Vaughn Charles) Caira Marie'
Searching for "Pole Paradise" by LJ da Joker feat. Juan Ramone...
No results found for: 'Pole Paradise LJ da Joker feat. Juan Ramone'
Searching for "Gratitude" by Betty Michaels...
Done.
Searching for "Feelin' Good" by Kingpyth0n...
No results found for: 'Feelin' Good Kingpyth0n'
Searching for "Boysenberry Kisses" by MutherFungus...
No results found for: 'Boysenberry Kisses MutherFungus'
Searching for "Clark Kent/Split" by G-therz...
Done.
Searching for "Open Door" by Jodie Coste...
Done.
Searching for "Milk and Honey" by Viridian Groove...
No results found for: 'Milk and Honey Viridian Groove'
Searching for "Cuban Pete" by Drew...
Done.
Searching for "O Holy Night" by Bobby Felder and Friends...
Done.
Searching for "The First Noel" by Leslie Odom Jr....
No results found for: 'The First Noel Leslie Odom

No results found for: 'For Your Love I Cry Lisabel'
Searching for "Stars Dance" by Lisabel...
No results found for: 'Stars Dance Lisabel'
Searching for "Just One Kiss" by Connie Stevens...
Done.
Searching for "It Just Ain't Right" by R B Calvert...
Done.
Searching for "No Regrets" by Ellen Pels...
Done.
Searching for "Someone to Love" by Tøm Hughes...
Done.
Searching for "Campbell Park 1997'" by Realcityfame...
No results found for: 'Campbell Park 1997' Realcityfame'
Searching for "Crippled" by Blue Hush...
Done.
Searching for "Day by Day" by Chris Klein and the Boulevards...
Done.
Searching for "Faith" by Ellen Pels...
Done.
Searching for "Underneath" by Christavie...
Done.
Searching for "Pink Monalisa" by Lanyking...
Done.
Searching for "Down From Tha Jump" by Realcityfame...
No results found for: 'Down From Tha Jump Realcityfame'
Searching for "Mirror" by MMM Acoustique...
No results found for: 'Mirror MMM Acoustique'
Searching for "Night Owl" by Attlus...
No results found for: 'Nig

No results found for: 'You Have to Start Somewhere Detlef Cordes'
Searching for "Mediocre" by D-Rad...
Done.
Searching for "Fool" by Aia...
Done.
Searching for "Sunset" by Drew Miles...
Done.
Searching for "Make Me Smile" by Diona Fox & Her Ragdolls...
No results found for: 'Make Me Smile Diona Fox & Her Ragdolls'
Searching for "Fun and Games" by Yasmin Marcelo...
Done.
Searching for "Lullaby" by Makassar Rocksteady...
No results found for: 'Lullaby Makassar Rocksteady'
Searching for "Pat a Cake" by Mikaela Persson & Árpád Solti...
No results found for: 'Pat a Cake Mikaela Persson & Árpád Solti'
Searching for "Snacks" by Elle Greyy...
Done.
Searching for "The Girl of My Best Friend" by Michael Bublé...
Done.
Searching for "You Can Turn Me On" by Embryo...
Done.
Searching for "Full Life" by Astrid...
Done.
Searching for "Into the World" by halie and the moon...
Done.
Searching for "Saturnian night" by Time Handlers Company...
Done.
Searching for "Worth It (Instrumental)" by Zoetic...
No

Done.
Searching for "Complicated" by Ajhani Azure...
Done.
Searching for "Before You Go" by Uncle Sixth...
Done.
Searching for "Everybody Loves Somebody" by Michael Bublé...
Done.
Searching for "Relapse" by Hishaam Bhimji...
No results found for: 'Relapse Hishaam Bhimji'
Searching for "Don't waste your time" by Singing Nikki...
Done.
Searching for "Never Told" by LSD...
Done.
Searching for "Short time" by Tamarae...
Done.
Searching for "The Ocean Pt.2" by Smalltape...
No results found for: 'The Ocean Pt.2 Smalltape'
Searching for "Are you mine" by IAMLLEYA...
Done.
Searching for "The Minuscule Masochist" by Stevie Spring...
No results found for: 'The Minuscule Masochist Stevie Spring'
Searching for "I Apologise (Re-Mastered 2011)" by Billy Eckstine...
No results found for: 'I Apologise (Re-Mastered 2011) Billy Eckstine'
Searching for "Sisters on the J Train" by Halley Elwell...
No results found for: 'Sisters on the J Train Halley Elwell'
Searching for "I'll Be Around" by Lisa Ekdahl...

Done.
Searching for "Yes Indeed" by Ray Charles and His Orchestra...
Done.
Searching for "You're the Top of My Hit Parade" by Matt Monro...
Done.
Searching for "WHO !?!?" by Jerré feat. Hobbes Duende...
No results found for: 'WHO !?!? Jerré feat. Hobbes Duende'
Searching for "Two Peas in a Pod" by Blaze, the Rebel...
Done.
Searching for "In A Sentimental Mood" by Mark Isham...
Done.
Searching for "Matter" by Drew The MC...
Done.
Searching for "Loose" by Connor Grant...
Done.
Searching for "Why You Worried 'bout Me?" by Rebirth Brass Band...
Done.
Searching for "I Love You" by Monita Tahalea...
Done.
Searching for "I Can Get It" by SoulBmoll...
Done.
Searching for "It's De-Lovely" by Manny Albam...
No results found for: 'It's De-Lovely Manny Albam'
Searching for "Absolutely Mad ! (Untouched)" by noPhiltr...
No results found for: 'Absolutely Mad ! (Untouched) noPhiltr'
Searching for "Wishes!" by Tekki...
Done.
Searching for "Airtight" by Delta Don...
Done.
Searching for "What If..?" by D